In [1]:
import time
from datetime import datetime

import mlflow
import requests
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
MLFLOW_API_URI = "http://localhost:5000/api/2.0"
ARTIFACT_ID = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
MODEL_NAME = "test-1"

In [3]:
def create_or_get_experiment():
    try:
        experiment_id = mlflow.create_experiment(name=MODEL_NAME)
    except:
        experiment = mlflow.get_experiment_by_name(name=MODEL_NAME)
        print(experiment)
        experiment_id = experiment.experiment_id
    mlflow.set_experiment(experiment_id=experiment_id)
    return experiment_id

In [4]:
n_estimators = 100
max_depth = 5

In [5]:
mlflow.set_tracking_uri("http://localhost:5000")

In [6]:
experiment_id = create_or_get_experiment()

<Experiment: artifact_location='/home/lhe/mlops/registry/artifacts/6', creation_time=1724265284866, experiment_id='6', last_update_time=1724265284866, lifecycle_stage='active', name='test-1', tags={}>


In [7]:
with mlflow.start_run() as run:
    iris = load_iris()
    X, y = iris.data, iris.target

    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=0.2,
                                                        random_state=42)
    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)

    # Train the model
    rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    mlflow.sklearn.log_model(rf, "model")
    mlflow.log_artifact("serving_input_payload.json",
                        artifact_path="model")

    # Print the results
    print(f"Run ID: {run.info.run_id}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

print(mlflow.get_run(run.info.run_id).to_dictionary())

2024/08/21 22:22:15 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/21 22:22:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run loud-hawk-739 at: http://localhost:5000/#/experiments/6/runs/ab7942dcc5ad42ff9d37475dcd9deaa3.
2024/08/21 22:22:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/6.


Run ID: ab7942dcc5ad42ff9d37475dcd9deaa3
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
{'info': {'artifact_uri': '/home/lhe/mlops/registry/artifacts/6/ab7942dcc5ad42ff9d37475dcd9deaa3/artifacts', 'end_time': 1724275335012, 'experiment_id': '6', 'lifecycle_stage': 'active', 'run_id': 'ab7942dcc5ad42ff9d37475dcd9deaa3', 'run_name': 'loud-hawk-739', 'run_uuid': 'ab7942dcc5ad42ff9d37475dcd9deaa3', 'start_time': 1724275332360, 'status': 'FINISHED', 'user_id': 'lhe'}, 'data': {'metrics': {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}, 'params': {'n_estimators': '100', 'max_depth': '5'}, 'tags': {'mlflow.user': 'lhe', 'mlflow.source.name': '/home/lhe/venv/mlops-demo/lib/python3.10/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.runName': 'loud-hawk-739', 'mlflow.log-model.history': '[{"run_id": "ab7942dcc5ad42ff9d37475dcd9deaa3", "artifact_path": "model", "utc_time_created": "2024-08-21 21:22:12.563545", "flavors": {"pytho